# Text als Muster

Aus Sicht einer Programmiersprache wie Python sind Texte nur Zeichenketten. Oft interessieren uns jedoch spezifische Informationen, die in Texten enthalten sind. Das kann z.B. der Autor eines Textes sein, oder das Datum der Veröffentlichung. Dabei stehen wir vor der Aufgabe, strukturierte Informationen (etwa einen Namen oder ein Datum) aus vergleichsweise unstrukturierten Daten wie Texten zu gewinnen. Dies ist das Feld der [Informationsextraktion](http://de.wikipedia.org/wiki/Informationsextraktion).

Wenn die Informationen im Text relativ gleichförming hinterlegt sind, lassen sich Regeln formulieren, nach denen die gewünsche Information gefunden werden kann. So steht etwa bei vielen Online-Artikeln eine Zeile wie:

> Dieser Beitrag wurde veröffentlicht am 20.1.2014 von Marie Müller.

Dieses Muster, das dem menschlichen Leser unmittelbar einleuchtet, muss dann in abstrakter Form beschrieben werden, um vom Computer angewandt werden zu können.

## Reguläre Ausdrücke

Hierfür unterstützen die meisten Programmiersprachen sogenannte »reguläre Ausdrücke«. Diese erlauben, Muster zu beschreiben und in Texten zu finden. Ein regulärer Ausdruck besteht in der Regel aus diesen Elementen:

*Was?*

Welche Zeichen sollen gefunden werden?

* Wörtliche Zeichen: Zeichen werden so gefunden, wie sie eingegeben werden, z.B. »`a`«.
* Zeichen aus einer bestimmten Menge, z.B. »`[aä]`« für a oder ä, »`[a-z]`« für die Kleinbuchstaben zwischen a und z.
* Zeichen aus einer vordefinierten Menge, z.B. »`\d`« für Zahlen *(decimal)* oder »`\w`« für Buchstaben und Zahlen *(word)*.
* Ein beliebiges Zeichen: »`.`«.

*Wie viel?*

Wie viele dieser Zeichen sollen gefunden werden?

* »`*`«: Keinmal oder mehrfach
* »`+`«: Einmal oder mehrfach
* »`?`«: Keinmal oder einmal
* »`{m,n}`«: Mindestens m-mal, höchstens n-mal, z.B. »`{2,4}`«.

*Möglichst viel oder möglichst wenig?*

In zweideutigen Fällen kann angegeben werden, ob möglichst viele oder möglichst wenige Zeichen gefunden werden sollen.

* »`*?`« oder »`+?`«: Möglichst wenig

Für eine ausführliche Beschreibung sei auf [Wikipedia](http://de.wikipedia.org/wiki/Regul%C3%A4rer_Ausdruck#Regul.C3.A4re_Ausdr.C3.BCcke_in_der_Praxis) und die [Python-Dokumentation](https://docs.python.org/3.4/library/re.html) verwiesen.

Im Teil zu [Text in Python](../00_Python/Text in Python.ipynb) wurde eine einfache Möglichkeit gezeigt, einen Text in Wörter zu zerlegen:

In [1]:
sample = '''Lieber Herr Kock,
liebe Kollegin, Frau Wanka,
meine Damen und Herren,
aber besonders: liebe Preisträgerinnen und Preisträger von „Jugend forscht“,'''
sample.split()

['Lieber',
 'Herr',
 'Kock,',
 'liebe',
 'Kollegin,',
 'Frau',
 'Wanka,',
 'meine',
 'Damen',
 'und',
 'Herren,',
 'aber',
 'besonders:',
 'liebe',
 'Preisträgerinnen',
 'und',
 'Preisträger',
 'von',
 '„Jugend',
 'forscht“,']

Das Problem hierbei ist, dass die einfache Regel „Wörter werden durch Leerzeichen getrennt“ im Falle von Satzzeichen nicht funktioniert, diese werden dann für einen Teil des Wortes gehalten. Exakter müsste die Regel heißen: „Wörter bestehen aus einem oder mehreren Buchstaben.“ Reguläre Ausdrücke ermöglichen es, eine solche Regel zu formulieren: `\w` steht für einen beliebigen Buchstaben (oder eine Zahl), `+` steht für eines oder mehrere dieser Zeichen.

Dazu muss zunächst das Modul `re` (für *regular expressions*) importiert werden:

In [2]:
import re

re.findall('\w+', sample)

['Lieber',
 'Herr',
 'Kock',
 'liebe',
 'Kollegin',
 'Frau',
 'Wanka',
 'meine',
 'Damen',
 'und',
 'Herren',
 'aber',
 'besonders',
 'liebe',
 'Preisträgerinnen',
 'und',
 'Preisträger',
 'von',
 'Jugend',
 'forscht']

Eine entsprechende Regel kann dann auch verwendet werden, um die Information zu Datum und Autor eines Textes zu extrahieren, wie im oben genannten Beispiel.

Das Datum besteht dabei aus Ziffern und Punkten. Eine sehr einfache Regel könnte also heißen: Finde eine Kette von Ziffern und Punkten. Für Ziffern steht das Muster »`\d`«, für den Punkt »`\.`« (weil »`.`« für ein beliebiges Zeichen steht). Die Menge aus beiden, also Ziffer *oder* Punkt, lässt sich als »`[\d\.]`« ausdrücken. Die einfache Regel wäre dann: »`[\d\.]+`« Da durch den vorangestellten Text »veröffentlicht am« sichergestellt wird, dass keine anderen Kombinationen aus Zahlen und Punkten (wie 3.000) gefunden werden, reicht diese einfache Regel.

Der Name besteht aus Zeichen, aber auch aus Leerzeichen (zwischen Vor- und Nachname). Das Ende des Namens kann aber am finalen Punkt erkannt werden. Die Regel für den Namen lautet also »`.+\.`«.

Die gesamte Regel lautet dann: »`veröffentlicht am [\d\.]+ von .+\.`«

Um die beiden Stellen zu identifiziert, die die gewünschte Information enthalten, können in regulären Ausdrücken Klammern verwendet werden. Dies erlaubt, später separat auf die Datumsangabe und den Autorennamen zurückzugreifen und sie vom umgebenden Text zu trennen:

In [3]:
text = 'Dieser Beitrag wurde veröffentlicht am 20.1.2014 von Marie Müller.'
pattern = 'veröffentlicht am ([\d\.]+) von (.+)\.'
match = re.search(pattern, text)
match.groups()

('20.1.2014', 'Marie Müller')

Im Gegensatz zu `re.findall()` gibt `re.search()` die Treffer also nicht direkt zurück, sondern speichert das Ergebnis in einer Form, die den gezielten Zugriff auf verschiedene Bestandteile erlaubt. `re.search()` kann dabei auch als Test verwendet werden: Wenn das Muster nicht auf den Text passt, wird statt eines Match-Objekts der Wert `None` zurückgegeben.

Manchmal kann es notwendig sein, die Suche etwas einzugrenzen. Wenn beispielsweise der Text einen weiteren Satz enthält, greift die Regel nicht mehr:

In [4]:
text = 'Dieser Beitrag wurde veröffentlicht am 20.1.2014 von Marie Müller. Alle Angaben ohne Gewähr.'
match = re.search(pattern, text)
match.groups()

('20.1.2014', 'Marie Müller. Alle Angaben ohne Gewähr')

Die Regel ist korrekt ausgewertet worden: Der Name wird interpretiert als ein oder mehrere beliebige Zeichen die mit einem Punkt enden. Entgegen der Intuition greift die Regel aber bis zum Punkt am Ende des *zweiten* Satzes. Um bei einer Regel, die mit »`+`« oder »`*`« mehrere Zeichen umfasst, nur den *kürzesten* möglichen Teil zu erfassen, kommt das »`?`« zum Einsatz:

In [5]:
pattern = 'veröffentlicht am ([\d\.]+) von (.+?)\.'
#                                             ^ hier
match = re.search(pattern, text)
match.groups()

('20.1.2014', 'Marie Müller')

## Datumsangaben

Die Datumsangabe ist im Beispiel als Zeichenkette erfasst worden: `'20.1.2014'`. Oft ist es wichtig, Datums- und Zeitangaben strukturierter zu erfassen. Wenn etwa Texte über die Zeit monatsweise analysiert werden sollen, muss man wissen, dass dieser Text im Januar erschienen ist. Das gilt auch, wenn Datumsangaben in natürlicher Sprache verfasst sind, also z.B. »27. Januar 2011«. Dazu muss die Zeichenkette in bestimmte Elemente, etwa Tag, Monat und Jahr, zerlegt werden.

In Python gibt es den Datentyp `date`, der den strukturierten Umgang mit Datumsangaben erlaubt:

In [6]:
import datetime
datetime.date.today()

datetime.date(2015, 10, 20)

In [7]:
date = datetime.date(2011, 1, 27)
date.year == 2011

True

Die Aufgabe besteht also zunächst darin, aus der Zeichenkette »27. Januar 2011« das Datum in strukturierter Form zu extrahieren. Dabei hilft es, dass wir wissen, dass hier zunächst der Tag steht, dann der Monat und dann das Jahr. Mit dieser Information lässt sich etwa das Jahr extrahieren.

In [8]:
date_string = '27. Januar 2011'
day, month, year = date_string.split()
int(year) == 2011

True

Für den Monat ist dies schwieriger, da er nicht als Zahl, sondern als Wort angegeben ist. Python verfügt über spezifische Funktionen für den Umgang mit Datumsangaben einschließlich Wochentagen und Monatsnamen. Dafür gibt es eine Art »reguläre Ausdrücke für Datumsangaben«. Um aus einer Zeichenkette, die einem bestimmten Aufbau folgt, ein Datum zu extrahieren, muss man nur das entsprechende Muster formulieren:

> Erst der Tag als Zahl, gefolgt von einem Punkt, dann der Monat als ausgeschriebenes Wort, dann das vierstellige Jahr.

Die entsprechenden Muster lassen sich in der [Python-Dokumentation](https://docs.python.org/2.7/library/datetime.html?highlight=strptime#strftime-strptime-behavior) oder auf [strftime.org](http://strftime.org/) nachlesen. Das Muster lautet dann:

In [9]:
pattern = '%d. %B %Y'

Die Funktion `strptime` (string parse time) erlaubt, dieses Muster für die Extraktion eines Datums anzuwenden. Dabei wird standardmäßig ein `datetime`-Objekt erstellt, dass nicht nur da Datum, sondern auch eine Zeitangabe enthält. Da dies hier nicht benötigt wird, lässt sich daraus aber auch nur das Datum extrahieren. Da Python zusätzlich wissen muss, dass die Monate auf Deutsch angegeben sind, muss die entsprechende Sprache aktiviert werden.

In [10]:
import locale
locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')

date_and_time = datetime.datetime.strptime(date_string, pattern)
date_and_time.date()

datetime.date(2011, 1, 27)

### Aufgabe

Extrahieren Sie aus den gegebenen Daten ein Datumsobjekt (`datetime.date`). Ggf. müssen Sie dazu mit einer Fallunterscheidung arbeiten.

In [11]:
data = [
    'Datum: 23.1.2011',
    'Datum: 12. Januar 2004',
    'Veröffentlicht am 5. September 1999.',
    'Publiziert 12.04.2012 von admin',
]

In [12]:
def extract_date(text):
    return None

[extract_date(text) for text in data]

[None, None, None, None]

## Ein Testfall

Ein Testdatensatz mit den auf der Seite der deutschen Bundesregierung veröffentlichten Reden enthält zwar Angaben zu Ort und Zeit, aber die Redner sind nicht explizit ausgewiesen. Sie werden allerdings in der Regel im Titel genannt. Ein Ansatz kann also sein, Angaben zur Rednerin bzw. zum Redner mittels regulärer Ausdrücke aus dem Titel zu extrahieren.

In [13]:
import pandas as pd
data = pd.read_csv('../Daten/Reden.csv', parse_dates=['date'], encoding='utf-8')

In [14]:
data.head()

,url,title,date,place,abstract,text
0,http://www.bundesregierung.de/Content/DE/Rede/...,Rede von Bundeskanzlerin Merkel beim Empfang d...,2015-09-30,Berlin,in Berlin\n,"Lieber Herr Kock,\nliebe Kollegin, Frau Wanka,..."
1,http://www.bundesregierung.de/Content/DE/Rede/...,Rede von Bundeskanzlerin Merkel beim Global Le...,2015-09-27,New York,NaN,"Sehr geehrter Herr Präsident Xi Jinping,\nsehr..."
2,http://www.bundesregierung.de/Content/DE/Rede/...,Rede von Bundeskanzlerin Merkel beim gemeinsam...,2015-09-26,New York,NaN,"Exzellenzen, \nmeine Damen und Herren, \n\nich..."
3,http://www.bundesregierung.de/Content/DE/Rede/...,Rede von Bundeskanzlerin Merkel beim Arbeitsmi...,2015-09-26,New York,NaN,"Sehr geehrter Herr Generalsekretär, lieber Ban..."
4,http://www.bundesregierung.de/Content/DE/Rede/...,Rede von Bundeskanzlerin Merkel beim High Leve...,2015-09-25,New York,NaN,"Sehr geehrter Herr Generalsekretär,\nsehr geeh..."


In [15]:
titles = data['title']
titles.head()

0    Rede von Bundeskanzlerin Merkel beim Empfang d...
1    Rede von Bundeskanzlerin Merkel beim Global Le...
2    Rede von Bundeskanzlerin Merkel beim gemeinsam...
3    Rede von Bundeskanzlerin Merkel beim Arbeitsmi...
4    Rede von Bundeskanzlerin Merkel beim High Leve...
Name: title, dtype: object

In [16]:
def extract_speaker(text):
    match = re.search('Rede von ([A-ZÄÖÜ]\w+(?:\s+[A-ZÄÖÜ]\w+)*)', text)
    if match:
        return match.group(1)
    else:
        return None
    
speakers = titles.apply(extract_speaker)  # entspricht [extract_speaker(title) for title in titles]
speakers.head()

0    Bundeskanzlerin Merkel
1    Bundeskanzlerin Merkel
2    Bundeskanzlerin Merkel
3    Bundeskanzlerin Merkel
4    Bundeskanzlerin Merkel
Name: title, dtype: object

In [17]:
speakers.value_counts()

Bundeskanzlerin Merkel                         227
Bundeskanzlerin Angela Merkel                  179
Kulturstaatsminister Bernd Neumann              89
Kulturstaatsministerin Monika Grütters          34
Kulturstaatsministerin Grütters                 31
Staatsminister Bernd Neumann                    30
Bundeskanzlerin Angela Merkel                   14
Staatsministerin Monika Grütters                 3
Monika Grütters                                  2
Bundeskanzlerin Merkel                           2
Kulturstaatsminister Bernd Neumann               2
Bundesminister Pofalla                           1
Kanzlerin Merkel                                 1
Staatsminister Bernd Neumann                     1
Bundesaußenminister Steinmeier                   1
Bundeslandwirtschaftsministerin Ilse Aigner      1
Joachim Gauck                                    1
Kulturstaatsministers Bernd Neumann              1
Staatsministerin Grütters                        1
BK                             

In [18]:
total = len(speakers)
empty = sum(speakers.isnull())
'Von {} {} erkannt und {} nicht erkannt'.format(total, total - empty, empty)

'Von 793 624 erkannt und 169 nicht erkannt'